## Birdsound Fieldguide for Stockton

In [ ]:
# Query Xeno-Canto for recordings of a specific bird species
# Filter recordings by type (call or song)
# Download and save them in downloads/{species}/{type}/ directories

In [3]:
import requests
import os

WESTERN_LOCATIONS = [
    "California", "CA", "Oregon", "OR", "Washington", "WA", "Nevada", "NV", "Arizona", "AZ",
    "Utah", "UT", "New Mexico", "NM", "Colorado", "CO", "Idaho", "ID", "Montana", "MT",
    "Wyoming", "WY", "Alaska", "AK"
]

def fetch_and_save_recordings(bird_name, base_dir="/Users/clarathomas/Desktop", limit=24):
    query_url = f"https://xeno-canto.org/api/2/recordings?query={bird_name.replace(' ', '+')}"
    downloaded = 0
    page = 1

    while downloaded < limit:
        response = requests.get(f"{query_url}&page={page}")
        if response.status_code != 200:
            print(f"Failed to fetch data on page {page}")
            break

        data = response.json()
        recordings = data.get("recordings", [])
        if not recordings:
            break

        for rec in recordings:
            if downloaded >= limit:
                break

            file_type = rec.get("type", "").lower()
            if file_type not in ["call", "song"]:
                continue

            location = rec.get("loc", "")
            if not any(loc.lower() in location.lower() for loc in WESTERN_LOCATIONS):
                continue

            if rec.get("q", "") not in ["A", "B"]:
                continue

            # Construct full audio URL
            file_url = rec['file'] if rec['file'].startswith("http") else f"https:{rec['file']}"
            print(f"\nDownloading {file_type} from {location}")
            print(f"Fetching from: {file_url}")

            # Generate a unique filename using XC ID
            xc_id = rec.get("id", "")
            file_name = f"XC{xc_id}_{file_type}.mp3"

            # Build target directory and path
            species_dir = os.path.join(base_dir, bird_name.replace(" ", "_"))
            type_dir = os.path.join(species_dir, file_type)
            os.makedirs(type_dir, exist_ok=True)
            mp3_path = os.path.join(type_dir, file_name)

            if os.path.exists(mp3_path):
                continue  # Skip existing files

            try:
                audio_data = requests.get(file_url, allow_redirects=True)
                if audio_data.status_code == 200:
                    content_type = audio_data.headers.get("Content-Type", "")
                    if "audio" not in content_type:
                        print(f"⚠️ Skipping non-audio file (type: {content_type})")
                        continue

                    with open(mp3_path, "wb") as f:
                        f.write(audio_data.content)

                    if os.path.getsize(mp3_path) == 0:
                        print(f"⚠️ Warning: Empty file downloaded: {mp3_path}")
                        os.remove(mp3_path)
                    else:
                        print(f"✅ Saved to: {mp3_path}")
                        downloaded += 1
                else:
                    print(f"❌ Failed to download (HTTP {audio_data.status_code})")

            except Exception as e:
                print(f"❌ Error downloading {file_url}: {e}")

        page += 1

# Example usage
fetch_and_save_recordings("Catharus ustulatus")  # Swainson's Thrush



Fetching from: https://xeno-canto.org/1012677/download
✅ Saved to: /Users/clarathomas/Desktop/Turdus_migratorius/song/XC1012677_song.mp3

Fetching from: https://xeno-canto.org/1010651/download
✅ Saved to: /Users/clarathomas/Desktop/Turdus_migratorius/song/XC1010651_song.mp3

Fetching from: https://xeno-canto.org/1005778/download
✅ Saved to: /Users/clarathomas/Desktop/Turdus_migratorius/song/XC1005778_song.mp3

Fetching from: https://xeno-canto.org/999808/download
✅ Saved to: /Users/clarathomas/Desktop/Turdus_migratorius/song/XC999808_song.mp3

Fetching from: https://xeno-canto.org/999643/download
✅ Saved to: /Users/clarathomas/Desktop/Turdus_migratorius/song/XC999643_song.mp3

Fetching from: https://xeno-canto.org/999642/download
✅ Saved to: /Users/clarathomas/Desktop/Turdus_migratorius/call/XC999642_call.mp3

Fetching from: https://xeno-canto.org/959619/download
✅ Saved to: /Users/clarathomas/Desktop/Turdus_migratorius/song/XC959619_song.mp3

Fetching from: https://xeno-canto.org/954